# HyTML - (X)HTML templator / generator for Hy

My environment for the sake of clarity:

In [1]:
(import hy sys)
(print "Hy version: " hy.__version__)
(print "Python" sys.version)

Hy version:  0.12.1
Python 3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]


## Import main macros

In [2]:
(require [hytml.macros [*]])
(import (hytml.macros (*)))

Let us also define a small helper macro to actually render html on the Jupyter Notebook rather than display generated html code as a string:

In [3]:
(import IPython)
(defmacro xml> [&rest code]
  `(IPython.display.HTML (html* ~@code)))

Then we are ready for the show!

## XML, HTML4 (HTML5 and XHTML coming)

At the moment macros from HyTML module includes `xml*` (`html*`) and `html4` macros to generate markup code. `xml*` is a generic generator which allows using any tag names and attributes. `html4` macro allows to use only html4 specified tag names. Complete chart is at the end of the document. 

Tags can be created with or without attributes, as well as with or without content. For example:

In [4]:
(print
 (xml* 
  (node) "\r\n"
  (node :attribute "value") "\r\n"
  (node :attribute "value" "Content")))

<node/>
<node attribute="value"/>
<node attribute="value">Content</node>


### Shorthand macro

`#㎖` (Square Ml) can be used as a short hand for generating xml/html/xhtml code.

In [5]:
#㎖(html
    (head (title "Page title"))
    (body (div "Page content" :class "container")))

'<html><head><title>Page title</title></head><body><div class="container">Page content</div></body></html>'

Same can be achieved with the more familiar macro notation `xml*`:

In [6]:
(xml*
  (html
    (head (title "Page title"))
    (body (div "Page content" :class "container"))))

'<html><head><title>Page title</title></head><body><div class="container">Page content</div></body></html>'

But `xml*` is especially convenient when generating multiple instances of code:

In [7]:
(xml* (p "Sentence 1") (p "Sentence 2") (p "Sentence 3"))

'<p>Sentence 1</p><p>Sentence 2</p><p>Sentence 3</p>'

Let us then render the code, not just printing it. This can be done via `xml>` helper macro defined ealier:

In [8]:
(xml> "Content is " (b king) !)

If validation of the html tag names is a concern, then one could use `html4` macro. In the example we try to use `time` element which is specifically used in html5 only:

In [9]:
(try
 (html4 (time))
 (except (e Exception)))

Traceback (most recent call last):
  File "C:\Users\phtep\Anaconda3\lib\site-packages\hy\macros.py", line 217, in macroexpand_1
    obj = wrap_value(m(*ntree[1:], **opts))
  File "C:\Users\phtep\OneDrive\Asiakirjat\GitHub\hytml/hytml/macros.hy", line 211, in _hy_anon_fn_25
    (.join "" (map parse-html4 code)))
  File "C:\Users\phtep\OneDrive\Asiakirjat\GitHub\hytml/hytml/macros.hy", line 157, in parse_html4
    (raise (Exception (% "Tag '%s' not meeting html4 specs" tag))))))
Exception: Tag 'time' not meeting html4 specs

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\phtep\Anaconda3\lib\site-packages\calysto_hy\kernel.py", line 89, in do_execute_direct
    _ast = hy_compile(tokens, '', root=ast.Interactive)
  File "C:\Users\phtep\Anaconda3\lib\site-packages\hy\compiler.py", line 2675, in hy_compile
    result = compiler.compile(tree)
  File "C:\Users\phtep\Anaconda3\lib\site-packages\hy\compiler.py", line 432, 

## Unquoting code

`html*` (same as `xml*`) is a macro so you can pass any code in it. See for example:

In [10]:
(html* (p "Sum: " (b (apply sum [[1 2 3 4]]))))

'<p>Sum: <b><apply>sum<[1, 2, 3, 4]/></apply></b></p>'

But you see, the result was not possibly what you expected. `html*` macro will interpret the first item of the _expression_ / _list_ as a name of the tag. Thus _apply_ becomes a tag name. Until the next _expression_ everything else is interpreted either as a content or a keyword.

<blockquote>`html*` doesn't give a dime of the used tag names. They can be anything, even processed names. Same applies to keywords, values, and the content. You can use more strict `html4` and `html5` macros to make sure, tag names are corresponding html4 [specifications](http://www.w3.org/TR/html401/index/elements.html).</blockquote>

However using `~` (unquote) symbol, `html*` macro behaviour can be stopped for a moment:

In [11]:
(html* (p "Sum: " (b ~(apply sum [[1 2 3 4]])) !))

'<p>Sum: <b>10</b>!</p>'

So the following expression after `~` will be evaluated and then result is returned back to the macro. And the rest of the code will be interpreted via macro. In this case it was just an exclamation mark.

<blockquote>Note that it is not mandatory to wrap strings with `""` if given input doesn't contain any spaces. You could also single quote simple non-spaced letter sequences. So `!` is same as `"!"` in this case.</blockquote>

Quoting and executing normal Hy code inside html gives almost unlimited possibility to use HyTML as a templating engine. Of cource there is also a risk to evaluate code that breaks the code execution. Plus uncontrolled template engine code may be a security risk.

## Unquote splice

In addition to unquote, one can handle lists and iterators with `~@` (unquote-splice) symbol. This is particularly handy when a list of html elements needs to be passed to the parent element. Take for example table head generation:

In [12]:
(html* 
 (table (thead
   (tr ~@(list-comp
         `(th :class (if (even? ~i) "even" "odd") ~label " " ~i)
         [[i label] (enumerate (* ["col"] 3))])))))

'<table><thead><tr><th class="even">col 0</th><th class="odd">col 1</th><th class="even">col 2</th></tr></thead></table>'

We should of course be able to use external source for the html. Let's try with csv data:

In [14]:
(xml> 
 (table :class "data"
   (caption "Contacts")
   (for-each
     [[idx row] (enumerate (.split (.read (open "data.csv" "r")) "\n"))]
     (if (pos? idx) `(do
         (tbody
            (for-each [item (.split "Mr.,John,07868785831" ",")]
              `(td ~item))))
         `(thead
            (for-each [item (.split "Title,Name,Phone" ",")]
              `(th ~item)))))))

## For-each

Let's do that once more but this time with a dedicated `for-each` macro that can be used to generate content on a loop. 

One more thing you should notice is how variables (col, row and cell) are referenced. Deeper on the nested quoted structure you are, unquoting must also be repeated accordingly.

In [16]:
(xml> 
  (table :class "data"
    (caption "Data table")
    (thead
      (tr
        (for-each [col ["col 1" "col 2" "col 3"]]
          `(th ~col))))
    (tbody 
     (for-each [row (range 3)]
       `(tr
         (for-each [cell (range 3)]
           `(td "cel " ~~row "." ~cell)))))
    (tfoot 
     (for-each [row (range 1)]
       `(tr
         (for-each [cell (range 3)]
           `(td "cel " ~~row "." ~cell)))))))

col 1,col 2,col 3
cel 0.0,cel 0.1,cel 0.2
cel 1.0,cel 1.1,cel 1.2
cel 2.0,cel 2.1,cel 2.2
cel 0.0,cel 0.1,cel 0.2


## HTML Specifications

`html*` does not care about the specifications. It is totally dummy about the naming conventions of the tags and their forms or attributes. `html4` macro will render tags as specified below. Using undefined tag will raise an error. Attributes are not validated however. One should use official [validator](http://validator.w3.org/) for such a task.

- Tag
- Tag name
- Forbidden (end tag)
- Omit (forbidden end tag plus omit short tag)

In [17]:
(xml> 
  (table :class "data"
    (caption "Data table")
    (thead
      (tr
        (for-each [col ["Tag" "Name" "Forbidden" "Omit"]]
          `(th ~col))))
    (tbody 
     (for-each [[idx1 row] (enumerate (do (import (hytml.specs (specs))) specs))]
       (do
        `(tr
         (td ~(.upper (.join "" (drop 2 row))))
         (td ~(get (get specs row) :name))
         (td ~(get (get specs row) :forbidden))
         (td ~(get (get specs row) :omit))))))))

Tag,Name,Forbidden,Omit
DIR,Directory list,False,False
AREA,Image map region,True,False
BDO,BiDi override,False,False
BASE,Document base URI,True,False
LI,List item,False,False
I,Italic text,False,False
OBJECT,Object,False,False
TT,Teletype text,False,False
HR,Horizontal rule,True,False
DL,Definition list,False,False


In [18]:
(IPython.display.HTML (.read (open "styles.css" "r")))

## The [MIT](http://choosealicense.com/licenses/mit/) License

Copyright (c) 2017 Marko Manninen